In [23]:
import fitz  # PyMuPDF
import os

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_folder = "data"

pdf_files = [f for f in os.listdir(pdf_folder) if f.startswith("orzeczenie") and f.endswith(".pdf")]

# Extract text
documents = []

for pdf_file in pdf_files:
    full_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(full_path)
    documents.append({"text": text, "source": pdf_file})

# Save corpus to file
with open("korpus.txt", "w", encoding="utf-8") as f:
    for doc in documents:
        f.write(f"### {doc['source']} ###\n")
        f.write(doc["text"])
        f.write("\n\n")

print("Zapisano korpus do pliku korpus.txt.")

Zapisano korpus do pliku korpus.txt.


In [24]:
texts = []
metadatas = []

for i, doc in enumerate(documents):
    texts.append(doc["text"])
    # Include both source filename and numeric ID in metadata
    metadatas.append({
        "source": doc["source"],
        "id": f"orzeczenie_{i}"
    })

print(f"Przygotowano {len(texts)} pełnych orzeczeń.")

Przygotowano 8 pełnych orzeczeń.


In [25]:
from sentence_transformers import SentenceTransformer

# Polish language model for legal documents
model = SentenceTransformer('ipipan/silver-retriever-base-v1.1')
embeddings = model.encode(texts, show_progress_bar=True, convert_to_numpy=True)

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


In [27]:
import chromadb
import os
import shutil

# Remove existing database if it exists
if os.path.exists("chroma_db"):
    shutil.rmtree("chroma_db")
    print("Removed existing database")

# Create directory for persistent storage
os.makedirs("chroma_db1", exist_ok=True)

# Create persistent Chroma client
chroma_client = chromadb.PersistentClient(path="chroma_db1")
chroma_client.delete_collection("orzeczenia")  # Delete existing collection if it exists
# Create collection
print("Creating collection 'orzeczenia'...")
collection = chroma_client.create_collection(name="orzeczenia")
print(f"Collection created successfully: {collection.name}")

# Add documents to the collection with proper metadata
print(f"Adding {len(texts)} documents to collection...")
collection.add(
    embeddings=embeddings.tolist(),
    documents=texts,
    metadatas=metadatas,  # Using the expanded metadata with source and ID
    ids=[f"orzeczenie_{i}" for i in range(len(texts))]
)
print(f"Successfully added {len(texts)} documents to collection")

Creating collection 'orzeczenia'...
Collection created successfully: orzeczenia
Adding 8 documents to collection...
Successfully added 8 documents to collection


In [46]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='ipipan/silver-retriever-base-v1.1')

chroma_vector_store = Chroma(client=chroma_client, collection_name="orzeczenia", embedding_function=embeddings)


In [47]:
import json
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama

# Initialize LLM
llm = Ollama(model="SpeakLeash/bielik-11b-v2.3-instruct:Q4_K_M")

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["context"],
    template="""
Na podstawie treści orzeczenia wypełnij następujące pola w formacie JSON.
Jeżeli nie ma informacji, wpisz `null`.
Format odpowiedzi:

{{
  "Typ_sprawy": "...",
  "Wiek_poszkodowanego": ...,
  "Czas_trwania_naruszenia_dni": ...,
  "Kwota_zadana": ...,
  "Procent_uszczerbku": ...,
  "Przyczynienie_procent": ...,
  "Typ_obrazenia_kategoria": "...",
  "Czy_cierpienie_znaczne": 0 lub 1,
  "Czy_skutki_trwale": 0 lub 1,
  "Czy_potrzeba_opieki": 0 lub 1,
  "Czy_wczesniejsze_swiadczenia": 0 lub 1,
  "Czy_diagnoza_psych": 0 lub 1,
  "Kwota_zasadzona_ostatecznie": ...
}}

Pracuj wyłącznie na podstawie treści orzeczenia.

Kontekst:
{context}

Odpowiedź:
"""
)

# Build LLM Chain
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Function to get the document by orzeczenie ID and generate structured response
def generate_orzeczenie_info(orzeczenie_id):
    # Retrieve the document by orzeczenie ID
    retrieved_document = chroma_vector_store.get(ids=[orzeczenie_id])
    print(f"Retrieved document: {retrieved_document}")

    if not retrieved_document or not retrieved_document.get('documents'):
        return {"error": f"Document with ID {orzeczenie_id} not found."}

    # Extract the content
    document_content = retrieved_document['documents'][0]

    # Generate output from LLM
    raw_response = llm_chain.run({"context": document_content})

    # Try parsing the response into JSON
    try:
        parsed_response = json.loads(raw_response)
        return parsed_response
    except json.JSONDecodeError:
        print(f"Warning: LLM returned invalid JSON:\n{raw_response}")
        return {"error": "Invalid JSON generated by LLM.", "raw_response": raw_response}

# Example usage
orzeczenie_id = "orzeczenie_0"  # Replace with the ID you want
response = generate_orzeczenie_info(orzeczenie_id)

print(json.dumps(response, indent=2, ensure_ascii=False))


Retrieved document: {'ids': ['orzeczenie_0'], 'embeddings': None, 'documents': ['Sygn. akt I ACa 703/15\nWYROK\nW IMIENIU RZECZYPOSPOLITEJ POLSKIEJ\nDnia 8 grudnia 2015 r.\nSąd Apelacyjny w Katowicach I Wydział Cywilny\nw składzie:\nPrzewodniczący :\nSSA Elżbieta Karpeta (spr.)\nSędziowie :\nSA Piotr Wójtowicz\nSA Joanna Naczyńska\nProtokolant :\nAnna Wieczorek\npo rozpoznaniu w dniu 8 grudnia 2015 r. w Katowicach\nna rozprawie\nsprawy z powództwa A. C., P. C. i D. C.\nprzeciwko  Towarzystwu (...) Spółce Akcyjnej w W.\no zapłatę\nna skutek apelacji powodów A. C. i D. C.\nod wyroku Sądu Okręgowego w Bielsku-Białej\nz dnia 22 maja 2015 r., sygn. akt I C 384/14,\nI. zmienia zaskarżony wyrok:\n1)  w punkcie 1/ i 4/ w ten sposób, że zasądza od pozwanej na rzecz powoda A. C. 60.000 (sześćdziesiąt tysięcy) złotych\nz ustawowymi odsetkami od 17 czerwca 2014 r., a w pozostałej części powództwo oddala,\n2)  w punkcie 2/ i 4/ w ten sposób, że zasądza od pozwanej na rzecz powódki D. C. 60.000 (sze

In [48]:
input = """Na podstawie treści orzeczenia wypełnij następujące pola w formacie JSON.
Jeżeli nie ma informacji, wpisz `null`.
Format odpowiedzi:

{{
  "Typ_sprawy": "...",
  "Wiek_poszkodowanego": ...,
  "Czas_trwania_naruszenia_dni": ...,
  "Kwota_zadana": ...,
  "Procent_uszczerbku": ...,
  "Przyczynienie_procent": ...,
  "Typ_obrazenia_kategoria": "...",
  "Czy_cierpienie_znaczne": 0 lub 1,
  "Czy_skutki_trwale": 0 lub 1,
  "Czy_potrzeba_opieki": 0 lub 1,
  "Czy_wczesniejsze_swiadczenia": 0 lub 1,
  "Czy_diagnoza_psych": 0 lub 1,
  "Kwota_zasadzona_ostatecznie": ...
}}

Pracuj wyłącznie na podstawie treści orzeczenia.

Kontekst:


'Sygn. akt I ACa 703/15\nWYROK\nW IMIENIU RZECZYPOSPOLITEJ POLSKIEJ\nDnia 8 grudnia 2015 r.\nSąd Apelacyjny w Katowicach I Wydział Cywilny\nw składzie:\nPrzewodniczący :\nSSA Elżbieta Karpeta (spr.)\nSędziowie :\nSA Piotr Wójtowicz\nSA Joanna Naczyńska\nProtokolant :\nAnna Wieczorek\npo rozpoznaniu w dniu 8 grudnia 2015 r. w Katowicach\nna rozprawie\nsprawy z powództwa A. C., P. C. i D. C.\nprzeciwko  Towarzystwu (...) Spółce Akcyjnej w W.\no zapłatę\nna skutek apelacji powodów A. C. i D. C.\nod wyroku Sądu Okręgowego w Bielsku-Białej\nz dnia 22 maja 2015 r., sygn. akt I C 384/14,\nI. zmienia zaskarżony wyrok:\n1)  w punkcie 1/ i 4/ w ten sposób, że zasądza od pozwanej na rzecz powoda A. C. 60.000 (sześćdziesiąt tysięcy) złotych\nz ustawowymi odsetkami od 17 czerwca 2014 r., a w pozostałej części powództwo oddala,\n2)  w punkcie 2/ i 4/ w ten sposób, że zasądza od pozwanej na rzecz powódki D. C. 60.000 (sześćdziesiąt tysięcy)\nzłotych z ustawowymi odsetkami od 17 czerwca 2014 r., a w pozostałej części powództwo oddala;\nII. oddala apelacje w pozostałej części;\nIII. zasądza od pozwanej na rzecz powodów A. C. i D. C.\npo 2.250 (dwa tysiące dwieście pięćdziesiąt) złotych z tytułu kosztów\npostępowania apelacyjnego.\nSSA Joanna Naczyńska\nSSA Elżbieta Karpeta\nSSA Piotr Wójtowicz\nSygn. akt I ACa 703/15\nUZASADNIENIE\nProwadząc połączone sprawy z powództw: D. C., A. C. i P. C. Sąd Okręgowy zasądził od pozwanego  Towarzystwa (...)\nSpółki Akcyjnej: na rzecz A. C. i D. C. po 40 000 zł. i na rzecz P. C. 20 000 zł. z ustawowymi odsetkami od 23 maja\n2015r., oddalił powództwa w pozostałej części i zniósł wzajemnie koszty procesu.\nRozstrzygnięcie to zapadło na tle niespornego między stronami stanu faktycznego, który nie był kwestionowany w\napelacji, w związku z czym nie wymaga powtórnego przytoczenia w uzasadnieniu Sądu Apelacyjnego, a to zgodnie z\nart. 387 § 21 k.p.c.\nPrzed omówieniem podstawy prawnej zapadłego orzeczenia, zgodnie z dyspozycją powołanego przepisu\nproceduralnego, wskazać jedynie należy, że zasądzone kwoty stanowią zadośćuczynienie za doznaną krzywdę\nwywołaną śmiercią J. C. – córki powodów A. i D. C., siostry powoda P. C.. Do wypadku komunikacyjnego, w\nktórym śmierć poniosła J. C. doszło w dniu 12 maja 2007r. z winy kierującego pojazdem mechanicznym, za którego\nodpowiedzialność cywilną ponosi pozwany.\nSąd Okręgowy wskazał, że podstawą prawną zasądzonych na rzecz każdego z powodów sum jest przepis art. 448\nk.c. umożliwiający zasądzenie odpowiedniego zadośćuczynienia w razie naruszenia dobra osobistego na rzecz osoby,\nktórej dobro osobiste zostało naruszone. Wskutek nagłej śmierci J. C. doszło do zerwania więzi rodzinnych i do\nnegatywnych następstw dla każdego z powodów z osobna, jak również dla funkcjonowania rodziny jako całości. Mając\npowyższe na uwadze Sąd Okręgowy ocenił, że odpowiednimi sumami zadośćuczynienia będą kwoty po 55 000 zł. na\nrzecz rodziców zmarłej i połowa tej kwoty tj. 28 000 zł. na rzecz jej brata. Podniósł nadto Sąd Okręgowy, że od śmierci J.\nC. upłynęło już 8 lat i minął dla powodów okres najintensywniejszych przeżyć emocjonalnych, stąd dalej idące żądania\npowodów ocenił jako zbyt wygórowane i podlegające oddaleniu. Odsetki od zasądzonych kwot uznał Sąd Okręgowy\nza uzasadnione od daty wyrokowania (a ściślej od dnia następnego po wydaniu wyroku), wskazując że przyznanie i\nokreślenie wysokości należnego zadośćuczynienia wymagało przeprowadzenia postępowania dowodowego, a „kwota\nzadośćuczynienia została przyznana wedle stanu na dzień zamknięcia rozprawy”.\nKoszty procesu wzajemnie zniesiono na podstawie art. 100 k.p.c.\nApelację od tego wyroku w części oddalającej żądania ponad kwoty 45 000 zł. oraz w odniesieniu do daty odsetek\nwnieśli powodowie A. C. i D. C.. Zarzucali naruszenie art. 448 w związku z art. 24 § 1 k.c. polegające na błędnej wykładni\ntego przepisu i przyjęciu, że suma 55 000 zł. z tytułu zadośćuczynienia na rzecz każdego z apelujących powodów jest\nkwotą odpowiednią do doznanej przez nich krzywdy. Naruszenie art. 481 w związku z art. 455 k.c. natomiast polegać\nmiało na przyjęciu błędnej daty wymagalności świadczenia.\nWnosili powodowie o zmianę zaskarżonego wyroku przez zasądzenie na rzecz każdego z nich dodatkowych kwot po\n45 000 zł. z odsetkami od 17 czerwca 2014r. Wnosili również o zasądzenie kosztów postępowania za obie instancje.\nPozwana Spółka wnosiła o oddalenie apelacji i zasądzenie kosztów postępowania apelacyjnego.\nSąd Apelacyjny zważył, co następuje.\nApelacja powodów okazała się uzasadniona.\nSąd Okręgowy w prawidłowo ustalonych okolicznościach dokonał nieprawidłowej oceny rozmiaru krzywdy powodów,\nwywołanej naruszeniem ich dóbr osobistych w postaci prawa do utrzymywania więzi rodzinnych, a tym samym\ndopuścił się zarzucanego w apelacji naruszenia art. 448 k.c. Zarówno z zeznań powodów, jak i opinii sądowo m-\npsychologicznej przeprowadzonej w toku postępowania wynikało, że nagła śmierć niespełna 20- letniej córki powodów\nwywołała bardzo silne negatywne przeżycia każdego z powodów, a także wpłynęła destrukcyjnie na związki rodzinne.\nPowodowie, stanowiący do wypadku zgodną, szczęśliwą rodzinę, od 2007r. stopniowo oddalają się od siebie, a\nnaprawa poważnie uszkodzonych więzi rodzinnych wymagałaby poddania się terapii. Bezpodstawnie przyjął Sąd\nOkręgowy, że upływ 8 lat od wypadku istotnie złagodził poczucie krzywdy. Wnioski opinii nie dają podstaw do\nuzasadnienia takiej oceny, a wręcz przeciwnie – wskazują na poważnie zaburzone relacje rodzinne mające źródło w\nprzeżywaniu faktu i okoliczności związanych z utratą córki. Zgodzić należało się z apelującymi, że wszechstronna i\nprawidłowa ocena wszystkich aspektów krzywdy powodów powinna prowadzić do zasądzenia wyższej, niż przyjął to\nSąd Okręgowy, sumy zadośćuczynienia. Kryteria odpowiedniej, w rozumieniu art. 448 k.c. sumy zadośćuczynienia\nspełniać będzie – w ocenie Sądu Apelacyjnego – kwota 75 000 zł. na rzecz każdego z powodów, co uzasadnia zmianę\nzaskarżonego wyroku na podstawie art. 386 § 1 k.p.c. przez zasądzenie na rzecz apelujących powodów kwot po 60 000\nzł., przy uwzględnieniu wypłaconej w postępowaniu likwidacyjnej kwoty 15 000 zł.\nUzasadniony był również zarzut apelacji dotyczący naruszenia przez Sąd Okręgowy art. 481 § 1 w związku z art. 455\nk.c. Sąd Okręgowy na uzasadnienie swego stanowiska w kwestii odsetek przytoczył stanowisko Sądu Najwyższego\nwynikające z wyroku z dnia 28 lutego 2011r. w sprawie I CSK 243/10. Teza trzecia tego orzeczenia w brzmieniu:\n„terminem, od którego należą się odsetki za opóźnienie w zapłacie zadośćuczynienia za krzywdę, może być więc,\nw zależności od okoliczności sprawy, zarówno dzień poprzedzający wyrokowanie o zadośćuczynieniu, jak i dzień\ntego wyrokowania.” została przez Sąd Okręgowy błędnie wykorzystana jako uzasadnienie stanowiska, że ustalenie\nwysokości zadośćuczynienia według stanu na dzień orzekania uzasadnia zasądzenie odsetek od dnia wyrokowania.\nTymczasem zarówno Sąd Najwyższy w wyroku z 28 lutego 2011r. (I CSK 243/10) jak i Sąd Apelacyjny we Wrocławiu\nw wyroku z dnia 10 lutego 2012r. (I ACa 1405/11), powołanym również w motywach skarżonego orzeczenia, orzekł\nzgoła odmiennie, niż uczynił to Sąd Okręgowy, o dacie początkowej odsetek od zasądzanego zadośćuczynienia. Jako\nugruntowane w orzecznictwie należy uznać stanowisko, że wyrok zasądzający zadośćuczynienie nie ma charakteru\nkonstytutywnego lecz deklaratoryjny, a roszczenie o zapłatę zadośćuczynienia ma charakter bezterminowy, stąd\nwezwanie do zapłaty przekształca je w roszczenie terminowe, a to zgodnie z art. 455 k.c., w myśl którego „jeżeli\ntermin spełnienia świadczenia nie jest oznaczony ani nie wynika z właściwości zobowiązania, świadczenie powinno\nbyć spełnione niezwłocznie po wezwaniu dłużnika do wykonania.” W uzasadnieniu wyroku z dnia 28 lutego\n2011 r. (I CSK 243/10) Sąd Najwyższy dokonał prezentacji stanowisk orzeczniczych w zakresie początkowej daty\nodsetek od zasądzanego zadośćuczynienia. Bez zbędnego powielania tych informacji zauważyć należy, że aktualnie w\norzecznictwie zarówno Sądu Najwyższego, jak i w przeważającej większości orzeczeń sądów apelacyjnych dominuje\nstanowisko, zgodnie z którym odsetki za opóźnienie w zapłacie zadośćuczynienia należne są uprawnionemu od\nterminu wyznaczonego przez art. 455 k.c. jeżeli w tym terminie zadośćuczynienie o którego wypłatę uprawniony\nwezwał zobowiązanego było mu należne. Zdarzać się mogą sytuacje, że w dacie wezwania zobowiązanego do wypłaty\nzadośćuczynienia nie było ono należne uprawnionemu w takiej wysokości jakiej żądał, a to w związku z faktem, że\nwysokość krzywdy może zmieniać się z upływem czasu. Jeżeli zatem w postępowaniu sądowym ustalone zostanie,\nże zadośćuczynienie w rozmiarze odpowiadającym sumie dochodzonej przez powoda należy się dopiero od dnia\nwyrokowania, odsetki od zasądzonego w takim przypadku zadośćuczynienia mogą się należeć dopiero od dnia\nwyrokowania. Nie jest to jednak tożsame z powołaną przez Sąd Okręgowy okolicznością, że określenia wysokości\nzadośćuczynienia dokonuje Sąd zawsze na podstawie stanu istniejącego w chwili wyrokowania, a to zgodnie z art.\n316 k.p.c. Jak wynika z wyroku Sądu Najwyższego z dnia 28 czerwca 2005r. w sprawie I CK 7/05 przyznawanie\nodsetek dopiero od dnia wyrokowania z powołaniem się na ustalanie wysokości zadośćuczynienia według stanu\nrzeczy istniejącego w chwili zamknięcia rozprawy i koncepcję waloryzacyjnego charakteru odsetek, pozbawione\njest aktualnie (tj. nie tylko w dacie orzeczenia Sądu Najwyższego ale i obecnie) doniosłości prawnej, bowiem od\nwielu lat nie występuje zjawisko deprecjacji waluty polskiej. Powodowie A. C. i D. C. wezwali pozwaną do zapłaty\nzadośćuczynienia w wysokości po 80 000 zł. w dniu 12 maja 2014r.Apelujący powołując się na 30 dniowy termin\ndo spełnienia świadczenia przez zakład ubezpieczeń wskazuje jednocześnie, że pozwana w dniu 16 czerwca 2014r.\npodjęła decyzję o odmowie zadośćuczynienia roszczeniu powodów w tej wysokości, stąd żądanie zasądzenia odsetek\nzostało zgłoszone od dnia 17 czerwca 2014r. Mając na uwadze przedstawione powyżej rozważania wskazać należy, że\nżądania powodów w zakresie wysokości zadośćuczynienia w kwotach po 80 000 zł. były uzasadnione w wysokości\nuznanej przez Sąd Apelacyjny jako adekwatne do rozmiaru krzywdy tj. w wysokości 75 000 zł. także w dniu kierowania\ndo pozwanego wezwania tj. w maju 2014r. Z tych względów skoro w terminie wynikającym z art. 14 ust. 1 ustawy\nz dnia 22 maja 2003r. o ubezpieczeniach obowiązkowych, Ubezpieczeniowym Funduszu Gwarancyjnym i Polskim\nBiurze Ubezpieczycieli Komunikacyjnych (t.j.Dz.U. 2013. 392) pozwana nie wypłaciła powodom żądanego zasadnie\nzadośćuczynienia, należne są powodom na podstawie art. 481 § 1 k.c. odsetki ustawowe od żądanej daty początkowej,\nczyli od 17 czerwca 2014r. W tym zakresie zatem zaskarżony wyrok również podlegał zmianie na podstawie art. 386\n§ 1 k.p.c.\nW pozostałym zakresie apelacja powodów nie była uzasadniona, a to z uwagi na fakt że przedstawione okoliczności nie\ndawały podstaw do uznania, że powodom należne miałoby być wyższe niż 75 000 zł. zadośćuczynienie, stąd apelacje\nw pozostałym zakresie zostały oddalone na mocy art. 385 k.p.c.\nWobec częściowego uwzględnienia apelacji koszty postępowania apelacyjnego zostały stosunkowo rozdzielone na\nmocy art. 100 k.p.c. Mając na uwadze zakres zaskarżenia zarówno w części roszczenia głównego jak i daty początkowej\nbiegu odsetek należało uznać, że powodowie wygrali etap postępowania apelacyjnego w około 48%. Powodowie\nponieśli koszty opłaty od apelacji w wysokości po 2451 zł. oraz wynagrodzenie ich pełnomocnika w wysokości 1800\nzł. Uwzględniając koszt wynagrodzenia pełnomocnika strony przeciwnej, łącznie koszty procesu w postępowaniu\napelacyjnym wyniosły 8502 zł. Ze stosunkowego rozdzielenia tych kosztów wynika zatem, że każdemu z powodów\npozwany winien zwrócić koszty postępowania apelacyjnego w wysokości 2250 zł.\nSSA Joanna Naczyńska SSA Elżbieta Karpeta SSA Piotr Wójtowicz\

Odpowiedź:"""

In [ ]:
{
  "Typ_sprawy": "O zadośćuczynienie za naruszenie dóbr osobistych",
  "Wiek_poszkodowanego": null,
  "Czas_trwania_naruszenia_dni": null,
  "Kwota_zadana": null,
  "Procent_uszczerbku": null,
  "Przyczynienie_procent": null,
  "Typ_obrazenia_kategoria": "Naruszenie dóbr osobistych (prawo do utrzymywania więzi rodzinnych)",
  "Czy_cierpienie_znaczne": 1,
  "Czy_skutki_trwale": 1,
  "Czy_potrzeba_opieki": null,
  "Czy_wczesniejsze_swiadczenia": null,
  "Czy_diagnoza_psych": 1,
  "Kwota_zasadzona_ostatecznie": {
    "A. C.": 60000,
    "D. C.": 60000
  }
}


# serwer rozbydowwywalny do 30k| zorientować się w temacie
# strukturyzowana generacja